In [ ]:
import os
import fitz  # PyMuPDF for PDFs
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes, serialization
from docx import Document

# Generate RSA Keys
def generate_keys():
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    public_key = private_key.public_key()

    # Save private key
    with open("private_key.pem", "wb") as f:
        f.write(private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.PKCS8,
            encryption_algorithm=serialization.NoEncryption()
        ))

    # Save public key
    with open("public_key.pem", "wb") as f:
        f.write(public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ))

    print("🔑 Keys generated successfully!")

# Load keys
def load_private_key():
    with open("private_key.pem", "rb") as f:
        return serialization.load_pem_private_key(f.read(), password=None)

def load_public_key():
    with open("public_key.pem", "rb") as f:
        return serialization.load_pem_public_key(f.read())

# Extract text from different file formats
def extract_text(file_path):
    _, ext = os.path.splitext(file_path)

    if ext == ".txt":
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()
    elif ext == ".pdf":
        doc = fitz.open(file_path)
        return "\n".join([page.get_text() for page in doc])
    elif ext == ".docx":
        doc = Document(file_path)
        return "\n".join([para.text for para in doc.paragraphs])
    else:
        raise ValueError("Unsupported file format!")

# Sign a document
def sign_document(file_path):
    private_key = load_private_key()
    document_text = extract_text(file_path).encode()

    signature = private_key.sign(
        document_text,
        padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
        hashes.SHA256()
    )

    with open("signature.sig", "wb") as f:
        f.write(signature)

    print(f"✅ Document signed successfully! Signature saved as 'signature.sig'.")

# Verify a document
def verify_signature(file_path, signature_path):
    public_key = load_public_key()
    
    try:
        document_text = extract_text(file_path).encode()

        with open(signature_path, "rb") as f:
            signature = f.read()

        public_key.verify(
            signature,
            document_text,
            padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
            hashes.SHA256()
        )

        print("✅ Signature is VALID! Document is authentic. 🎉")
    except Exception as e:
        print("❌ Signature is INVALID! Document may be altered or tampered with.")
        print(f"Error: {e}")

# CLI Menu
def main():
    if not os.path.exists("private_key.pem") or not os.path.exists("public_key.pem"):
        print("🔑 Generating keys...")
        generate_keys()

    while True:
        print("\n1. Sign a document")
        print("2. Verify a signature")
        print("3. Exit")

        choice = input("Enter choice: ")

        if choice == "1":
            file_path = input("Enter document path: ").strip()
            if os.path.exists(file_path):
                sign_document(file_path)
            else:
                print("❌ File not found!")

        elif choice == "2":
            file_path = input("Enter original document path: ").strip()
            sig_path = input("Enter signature file path: ").strip()

            if os.path.exists(file_path) and os.path.exists(sig_path):
                verify_signature(file_path, sig_path)
            else:
                print("❌ File(s) not found!")

        elif choice == "3":
            print("Exiting... 🚀")
            break

        else:
            print("Invalid choice! Try again.")

if __name__ == "__main__":
    main()
1

🔑 Generating keys...
🔑 Keys generated successfully!

1. Sign a document
2. Verify a signature
3. Exit


Enter choice:  1
Enter document path:  "C:\Users\ishan\jbhdhjds.pdf"


❌ File not found!

1. Sign a document
2. Verify a signature
3. Exit


Enter choice:  1
Enter document path:  C:\Users\ishan\jbhdhjds.pdf


✅ Document signed successfully! Signature saved as 'signature.sig'.

1. Sign a document
2. Verify a signature
3. Exit


...

✅ Signature is VALID! Document is authentic. 🎉

1. Sign a document
2. Verify a signature
3. Exit


Enter choice:  1
Enter document path:  C:\Users\ishan\OneDrive\Documents\Hirva\6th SEM\bis proj2.docx


✅ Document signed successfully! Signature saved as 'signature.sig'.

1. Sign a document
2. Verify a signature
3. Exit


Enter choice:  2
Enter original document path:  C:\Users\ishan\OneDrive\Documents\Hirva\6th SEM\bis proj2.docx
Enter signature file path:  signature.sig


❌ Signature is INVALID! Document may be altered or tampered with.
Error: 

1. Sign a document
2. Verify a signature
3. Exit
